In [13]:
pip install pandas openpyxl matplotlib


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Font
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.worksheet.page import PageMargins
from openpyxl import load_workbook

# Load the data excel file and select the appropriate sheet
file_path = "B+M INV WK 25.xlsx"
df_data = pd.read_excel(file_path, sheet_name='SITE INV OH')

# Extract only the first 8 digits from the 'Article' column
df_article_trimmed = df_data[['GENERIC ID', 'SITE', 'INV']].copy()
df_article_trimmed['Article_8digit'] = df_article_trimmed['GENERIC ID'].astype(str).str.extract(r'(\d{1,8})')

# Drop the original 'Article' column
df_article_trimmed.drop(columns=['GENERIC ID'], inplace=True)

# Drop rows where INV is exactly 0
df_article_trimmed = df_article_trimmed[df_article_trimmed['INV'] != 0]

# Display the result
df_article_trimmed

,SITE,INV,Article_8digit
0,3801,5,13888176
1,3801,6,13888174
2,3801,9,13889003
3,3801,2,13888211
4,3801,1,13890243
...,...,...,...
8241,3845,34,13934394
8242,3845,1,13934373
8243,3845,19,14086901
8244,3845,3,13934323


In [ ]:
# === Load NMS list and clean GENERIC ID column ===
nms_path = "MD LIST B+M.xlsx"
df_nms_full = pd.read_excel(nms_path, sheet_name='NEW MSRP ', header=0)

# Extract width code (e.g., WM, WW, MM) into new column
# Width code is directly followed by digits, so use a lookahead
width_code_pattern = r'_(W[MWXS]|M[MWXS])(?=\d+)'
df_nms_full['WIDTH CODE'] = df_nms_full['STYLE NAME - WIDTH - COL ID'].str.extract(width_code_pattern)

# Remove width code from style column (keep the digits)
df_nms_full['STYLE NAME - WIDTH - COL ID'] = df_nms_full['STYLE NAME - WIDTH - COL ID'].str.replace(
    width_code_pattern, '', regex=True
)

# Strip extra whitespace
df_nms_full['STYLE NAME - WIDTH - COL ID'] = df_nms_full['STYLE NAME - WIDTH - COL ID'].str.strip()

# Reorder columns to place 'WIDTH CODE' right after 'STYLE NAME - WIDTH - COL ID'
cols = df_nms_full.columns.tolist()
style_idx = cols.index("STYLE NAME - WIDTH - COL ID")
cols.insert(style_idx + 1, cols.pop(cols.index("WIDTH CODE")))
df_nms_full = df_nms_full[cols]

# Rename columns
df_nms_full = df_nms_full.rename(columns={
    "STYLE NAME - WIDTH - COL ID": "STYLE NAME - COL ID",
    "WIDTH CODE": "WIDTH"
})

# Map width codes to readable values
width_mapping = {
    "WM": "Med", "WW": "Wide", "WX": "Extra Wide", "WS": "Slim",
    "MM": "Med", "MW": "Wide", "MX": "Extra Wide", "MS": "Slim"
}
df_nms_full["WIDTH"] = df_nms_full["WIDTH"].replace(width_mapping)

# Clean GENERIC ID to just first 8 digits
df_nms_full['GENERIC ID'] = (
    df_nms_full['GENERIC ID'].astype(str)
    .str.replace(r"\.0$", "", regex=True)
    .str.extract(r'(\d{1,8})')[0]
)

df_nms_full

,PC NAME,GENERIC ID,MC ID,STS#,COLLECTION,STYLE NAME - COL ID,WIDTH,NEW MSRP
0,Ladies Boots,13889535,MC14008,STS81732,WWW,SALTWATERCOR000001,Med,100
1,Ladies Boots,13890252,MC14008,STS91176,WWW,SALTWATERCOR000236,Med,110
2,Ladies Boots,13889533,MC14008,STS91175,WWW,SALTWATERCOR000251,Med,110
3,Ladies Boots,13889831,MC14008,STS91175,WWW,SALTWATERCOR000251,Wide,110
4,Ladies Boots,13934018,MC14008,STS87750,WWW,TORRENT000001,Med,90
...,...,...,...,...,...,...,...,...
154,Men Shoes,13881746,MC14020,STS25697,WWW,WAVEDRIVER1E000260,Med,100
155,Men Shoes,13881784,MC14020,STS25697,WWW,WAVEDRIVER1E000260,Wide,100
156,Men Shoes,13881743,MC14020,STS25766,WWW,WAVEDRIVERVN000001,Med,100
157,Men Shoes,13881745,MC14020,STS25746,WWW,WAVEDRIVERVN000260,Med,100


In [ ]:
# Merge on cleaned 8-digit article and Generic_ID ===
merged_full_df = pd.merge(
    df_article_trimmed,
    df_nms_full,
    how='inner',
    left_on='Article_8digit',
    right_on='GENERIC ID'
)

# Show result ===
merged_full_df

,SITE,INV,Article_8digit,PC NAME,GENERIC ID,MC ID,STS#,COLLECTION,STYLE NAME - COL ID,WIDTH,NEW MSRP
0,3801,20,13889535,Ladies Boots,13889535,MC14008,STS81732,WWW,SALTWATERCOR000001,Med,100
1,3801,22,13890252,Ladies Boots,13890252,MC14008,STS91176,WWW,SALTWATERCOR000236,Med,110
2,3801,18,13889533,Ladies Boots,13889533,MC14008,STS91175,WWW,SALTWATERCOR000251,Med,110
3,3801,36,14086988,Ladies Shoes,14086988,MC14001,10410879,ALDO,ANGELFISH000236,Med,110
4,3801,38,14086986,Ladies Shoes,14086986,MC14001,10410877,ALDO,ANGELFISH000932,Med,110
...,...,...,...,...,...,...,...,...,...,...,...
1497,3845,36,13881902,Men Shoes,13881902,MC14021,STS41110,WWW,HEADSAIL000020,Med,115
1498,3845,27,13888902,Men Shoes,13888902,MC14021,10528703,WWW,SEAKITESPRT000020,Med,115
1499,3845,38,13890307,Men Shoes,13890307,MC14021,10528695,WWW,SEAKITESPRT000230,Med,115
1500,3845,19,14086901,Men Shoes,14086901,MC14020,10413593,ALDO,WAVEDRIVER000007,Med,100


In [ ]:

# === Clean SITE column ===
merged_full_df['SITE'] = merged_full_df['SITE'].astype(str).str.strip().fillna('Unknown')

# Drop helper column
if 'Article_8digit' in merged_full_df.columns:
    merged_full_df.drop(columns=['Article_8digit'], inplace=True)

# === Group by SITE ===
grouped_dfs = {
    site: group.drop_duplicates().reset_index(drop=True)
    for site, group in merged_full_df.groupby('SITE')
}

# === Total markdown lines ===
total_md_lines = len(df_nms_full)
print(f"Total markdown lines: {total_md_lines}")

# === Build summary dataframe ===
summary = pd.DataFrame([
    {
        "SITE": site,
        "Lines in Site List": len(df),  # helper
        "Total Markdown Lines": total_md_lines,  # helper
        "Markdown Coverage": f"{len(df)} of {total_md_lines}",
        "Coverage %": f"{round(len(df) / total_md_lines * 100)}%"
    }
    for site, df in grouped_dfs.items()
])

# === Keep only necessary columns and sort ===
summary = summary[["SITE", "Markdown Coverage", "Coverage %"]]
summary["% Sort"] = summary["Coverage %"].str.replace('%', '').astype(int)
summary = summary.sort_values("% Sort", ascending=False).drop(columns=["% Sort"]).reset_index(drop=True)

# === Add AVERAGE row ===
coverage_values = summary["Coverage %"].str.replace('%', '').astype(int)
average_coverage = round(coverage_values.mean())

average_row = pd.DataFrame([{
    "SITE": "AVERAGE",
    "Markdown Actual List": "-",
    "Actual MD %": f"{average_coverage}%"
}])

summary = pd.concat([summary, average_row], ignore_index=True)

# === Export to Excel ===
output_path = "markdown_summary_by_site.xlsx"
summary.to_excel(output_path, index=False)

# === Format Excel ===
wb = load_workbook(output_path)
ws = wb.active

# Add table with correct range
table = Table(displayName="MarkdownCoverageSummary", ref=f"A1:C{ws.max_row}")
style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=False)
table.tableStyleInfo = style
ws.add_table(table)

# Bold headers
for cell in ws[1]:
    cell.font = Font(bold=True)

# Auto-adjust column widths & center alignment
for col in ws.columns:
    max_len = 0
    col_letter = get_column_letter(col[0].column)

    for cell in col:
        cell.alignment = Alignment(horizontal='center', vertical='center')
        if cell.value:
            max_len = max(max_len, len(str(cell.value)))

    ws.column_dimensions[col_letter].width = max_len + 2

# Manually widen SITE column (column A)
ws.column_dimensions["A"].width = max(ws.column_dimensions["A"].width, 10)

# Bold the AVERAGE row
for cell in ws[ws.max_row]:
    cell.font = Font(bold=True)

# Save workbook
wb.save(output_path)

print(f"Summary exported: {output_path}")

Total markdown lines: 159
Summary exported: markdown_summary_by_site.xlsx


In [19]:
# === Desired column order ===
desired_order = [
    "SITE", "PC NAME", "MC ID", "STYLE NAME - COL ID", "COL NAME", "WIDTH", "STS#",
    "GENERIC ID", "INV", "MSRP", "B+M $", "COLLECTION", "ACTION TAKEN"
]

# === Output path ===
output_path = "Per_Site_Markdown_For_Jonathan.xlsx"

# === Loop through grouped_dfs ===
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for site, df_site in grouped_dfs.items():
        # Reorder columns (keep extras at end)
        cols_to_keep = [col for col in desired_order if col in df_site.columns]
        extra_cols = [col for col in df_site.columns if col not in cols_to_keep]
        df_site = df_site[cols_to_keep + extra_cols]

        # Sort by first 4 columns
        sort_keys = [col for col in desired_order[:4] if col in df_site.columns]
        df_site.sort_values(by=sort_keys, inplace=True)

        # Write to sheet
        sheet_name = f"Site_{str(site)}"[:31]
        df_site.to_excel(writer, sheet_name=sheet_name, index=False)
        ws = writer.sheets[sheet_name]

        num_rows, num_cols = df_site.shape
        last_col_letter = get_column_letter(num_cols)

        # Convert range to Excel Table
        table_range = f"A1:{last_col_letter}{num_rows + 1}"
        table = Table(displayName=f"Table_{site}".replace(" ", "_"), ref=table_range)
        style = TableStyleInfo(name="TableStyleMedium2", showFirstColumn=False,
                               showLastColumn=False, showRowStripes=True, showColumnStripes=False)
        table.tableStyleInfo = style
        ws.add_table(table)

        # Center-align all data rows
        for row in ws.iter_rows(min_row=2, max_row=num_rows + 1, max_col=num_cols):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')

        # Set column widths and hide "MC ID"
        for col_idx, column_cells in enumerate(ws.columns, 1):
            col_letter = get_column_letter(col_idx)
            header = str(column_cells[0].value).strip()

            if header.upper() == "MC ID":
                ws.column_dimensions[col_letter].hidden = True
                continue

            # Currency columns wider
            if header.upper() in ["MSRP", "B+M $"]:
                ws.column_dimensions[col_letter].width = 18
            else:
                max_len = max((len(str(cell.value)) for cell in column_cells if cell.value), default=0)
                ws.column_dimensions[col_letter].width = max(max_len + 2, 12)

        # Format MSRP and B+M $ columns as currency
        for col_idx, col_name in enumerate(df_site.columns, start=1):
            if col_name.upper() in ["MSRP", "B+M $"]:
                for row in ws.iter_rows(min_row=2, max_row=num_rows + 1, min_col=col_idx, max_col=col_idx):
                    for cell in row:
                        cell.number_format = '"$"#,##0.00'

        # Page setup for printing
        ws.page_setup.orientation = 'landscape'
        ws.page_setup.fitToPage = True
        ws.page_setup.fitToWidth = 1
        ws.page_setup.fitToHeight = 0
        ws.page_margins = PageMargins(
            left=0.25, right=0.25, top=0.5, bottom=0.5, header=0.25, footer=0.25
        )
        ws.print_title_rows = '1:1'  # Repeat header row when printing

print(f"Excel file with per-site sheets saved to: {output_path}")

Excel file with per-site sheets saved to: Per_Site_Markdown_For_Jonathan.xlsx


In [20]:
import os
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

# === Settings ===
output_dir = "site_pdfs"
os.makedirs(output_dir, exist_ok=True)

currency_cols = ["NEW MSRP", "B+M $"]
desired_order = [
    "SITE", "PC NAME", "MC ID", "STYLE NAME - COL ID", "COL NAME", "WIDTH", "STS#",
    "GENERIC ID", "INV", "COLLECTION", "NEW MSRP"
]

rows_per_page = 40
landscape_figsize = (16, 10)  # width x height (landscape)

# === PDF Generation Loop ===
for site, df_site in grouped_dfs.items():
    df_site = df_site.copy()

    # Reorder columns
    cols_to_use = [col for col in desired_order if col in df_site.columns]
    extra_cols = [col for col in df_site.columns if col not in cols_to_use]
    df_site = df_site[cols_to_use + extra_cols]

    # Sort by first 4 columns
    sort_cols = [col for col in ["SITE", "PC NAME", "STYLE NAME - COL ID", "WIDTH"] if col in df_site.columns]
    df_site.sort_values(by=sort_cols, inplace=True)

    # Format currency
    for col in currency_cols:
        if col in df_site.columns:
            df_site[col] = df_site[col].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else "")

    # Paginate
    total_pages = math.ceil(len(df_site) / rows_per_page)
    site_str = str(site).replace("/", "_")
    pdf_path = os.path.join(output_dir, f"Site_{site_str}.pdf")

    with PdfPages(pdf_path) as pdf:
        for page in range(total_pages):
            start_row = page * rows_per_page
            end_row = start_row + rows_per_page
            df_chunk = df_site.iloc[start_row:end_row]

            fig, ax = plt.subplots(figsize=landscape_figsize)
            ax.axis("off")

            table = ax.table(
                cellText=df_chunk.values,
                colLabels=df_chunk.columns,
                loc='center',
                cellLoc='center',
                colWidths=[
                    0.20 if col == "STYLE NAME - COL ID" else
                    0.12 if col == "COL NAME" else
                    0.07  # default width for other columns
                    for col in df_chunk.columns
                    ]
            )

            table.auto_set_font_size(False)
            table.set_fontsize(8)
            table.scale(1.2, 1.2)

            # Bold header row and banded rows
            for (row, col), cell in table.get_celld().items():
                if row == 0:
                    cell.set_fontsize(9)
                    cell.set_text_props(weight='bold')
                    cell.set_facecolor("#d9d9d9")
                elif row % 2 == 1:
                    cell.set_facecolor("#f2f2f2")

            # Title
            ax.set_title(f"Markdown Report – Site {site} (Page {page + 1} of {total_pages})", fontsize=14, pad=20)

            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)

print(f" PDF files saved for each site in: {output_dir}")

 PDF files saved for each site in: site_pdfs
